<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Loading" data-toc-modified-id="Data-Loading-1">Data Loading</a></span></li><li><span><a href="#Figure3:-Streamflow-statistic" data-toc-modified-id="Figure3:-Streamflow-statistic-2">Figure3: Streamflow statistic</a></span></li><li><span><a href="#Quantization-methods" data-toc-modified-id="Quantization-methods-3">Quantization methods</a></span></li><li><span><a href="#Summary-of-Results" data-toc-modified-id="Summary-of-Results-3">Summary of Results </a></span><ul class="toc-item"><li><span><a href="#Figure4:-Histograms-of-different-quantization-methods-for-Station12" data-toc-modified-id="#Figure4:-Histograms-of-different-quantization-methods-for-Station12-3.1">Figure4: Histograms of different quantization methods for Station12</a></span></li><li><span><a href="#Figure5:-Histograms-of-different-quantization-methods-for-Station1" data-toc-modified-id="#Figure5:-Histograms-of-different-quantization-methods-for-Station1-3.2">Figure5: Histograms of different quantization methods for Station1</a></span></li><li><span><a href="#Figure6:-Number-and-center-of-bins-in-different-quantization-methods-for-Station1" data-toc-modified-id="Figure6-3.3">Figure6: Number and center of bins in different quantization methods for Station1</a></span></li><li><span><a href="#Figure7:-Number-and-center-of-bins-in-different-quantization-methods-for-Station12" data-toc-modified-id="Figure7-3.4">Figure7: Number and center of bins in different quantization methods for Station12</a></span></li><li><span><a href="#Table1:-Comparison-of-data-representativeness-with-Surprise-score" data-toc-modified-id="Table1-3.5">Table1: Comparison of data representativeness with Surprise score</a></span></li><li><span><a href="#Table2:-Comparison-of-data-representativeness-with-divergence-score" data-toc-modified-id="Table2-3.6">Table2: Comparison of data representativeness with divergence score</a></span></li><li><span><a href="#Table3:-Optimal-gauges-order-with-different-quantization-methods-and-objective-functions" data-toc-modified-id="Table3-3.7">Table3: Optimal gauges order with different quantization methods and objective functions </a></span></li><li><span><a href="#Table4:-Evolution-of-multivariate-joint-entropy-based-on-maxJE-method" data-toc-modified-id="Table4-3.8">Table4: Evolution of multivariate joint entropy based on maxJE method </a></span></li></ul></li></ul></div>

# Data Loading

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from glob import glob
import scipy.stats
import matplotlib.pyplot as plt
import math
from matplotlib import ticker as mticker
originaldata=pd.read_csv('rawdata.csv')
originaldata

ModuleNotFoundError: No module named 'seaborn'

# Figure3: Streamflow statistic

In [ ]:
df1=originaldata
df1.columns = ['ID:1', 'ID:2','ID:3','ID:4','ID:5','ID:6',
               'ID:7','ID:8','D:9','ID:10','ID:11','ID:12']
# Set up the matplotlib figure
sns.set(style="whitegrid",font_scale=1.2,)
fig, ax = plt.subplots(figsize=(25,10))
sns.despine(left=True, bottom=True)
sns.violinplot(data=df1 ,cut=0,scale= 'count',inner='box',palette="Set1",label='big',fontsize=12)
ax.set_xlabel('Station IDs',fontweight='bold',fontsize=30)
ax.set_ylabel('Monthly streamflow',fontweight='bold',fontsize=30)
ax.set_title(r'Monthly streamflow statistics of Brazos River $\left(\frac{m^3}{s}\right)$',fontweight='bold',fontsize=22)

yposlist = df1.max()+80
mean=df1.mean()
SD=originaldata.std()
xposlist = range(len(yposlist))

stringlist = ["mean="+str(float("{0:.2f}".format(mean[0]))),"mean="+str(float("{0:.2f}".format(mean[1]))),"mean="+str(float("{0:.2f}".format(mean[2]))),
              "mean="+str(float("{0:.2f}".format(mean[3]))),"mean="+str(float("{0:.2f}".format(mean[4]))),"mean="+str(float("{0:.2f}".format(mean[5]))),
              "mean="+str(float("{0:.2f}".format(mean[6]))),"mean="+str(float("{0:.2f}".format(mean[7]))),"mean="+str(float("{0:.2f}".format(mean[8]))),
              "mean="+str(float("{0:.2f}".format(mean[9]))),"mean="+str(float("{0:.2f}".format(mean[10]))),"mean="+str(float("{0:.2f}".format(mean[11])))]

stringlist2 = ["SD="+str(float("{0:.2f}".format(SD[0]))),"SD="+str(float("{0:.2f}".format(SD[1]))),"SD="+str(float("{0:.2f}".format(SD[2]))),
              "SD="+str(float("{0:.2f}".format(SD[3]))),"SD="+str(float("{0:.2f}".format(SD[4]))),"SD="+str(float("{0:.2f}".format(SD[5]))),
              "SD="+str(float("{0:.2f}".format(SD[6]))),"SD="+str(float("{0:.2f}".format(SD[7]))),"SD="+str(float("{0:.2f}".format(SD[8]))),
              "SD="+str(float("{0:.2f}".format(SD[9]))),"SD="+str(float("{0:.2f}".format(SD[10]))),"SD="+str(float("{0:.2f}".format(SD[11])))]

for i in range(len(stringlist)):
    ax.text(xposlist[i], yposlist[i], stringlist2[i],
    horizontalalignment='center',
    verticalalignment='top',fontsize=15,fontweight='bold')
    ax.text(xposlist[i], yposlist[i]+60, stringlist[i],
    horizontalalignment='center',
    verticalalignment='top',fontsize=15,fontweight='bold')
plt.show()
fig.savefig("statistics_v7.png",dpi=500, bbox_inches="tight", pad_inches=.5,transparent=True)

# Quantization methods

In [ ]:
# Data quantization
from Info_theory_function_lists import km_global_quantization
from Info_theory_function_lists import km_local_quantization
from Info_theory_function_lists import eqdistant_global_quantization
from Info_theory_function_lists import eqdistant_local_quantization
from Info_theory_function_lists import floor_global_quantization

a_f=150 # 150(m^3/s) for  a_f parameter defined in the paper

# eq_global_quantized8 returns a series of bin numbers while eq_global_quantized8_values returns a series of quantized values (representing center of the bins that original data falls in)
eq_global_quantized8,eq_global_quantized8_values,NA,eq_global_edges8=eqdistant_global_quantization(originaldata,8)
eq_global_quantized14,eq_global_quantized14_values,NA,eq_global_edges14=eqdistant_global_quantization(originaldata,14)
km_global_quantized8,km_global_quantized8_values,NA,km_global_edges8=km_global_quantization(originaldata,8)
km_global_quantized14,km_global_quantized14_values,NA,km_global_edges14=km_global_quantization(originaldata,14)
floor_global_quantized14,floor_global_quantized14_values,NA,floor_global_edges14=floor_global_quantization(originaldata,a_f)

Station_list=originaldata.columns
eq_local_quantized8= pd.DataFrame(np.zeros(240*12, dtype='f').reshape(240,12),columns=Station_list)
km_local_quantized8= pd.DataFrame(np.zeros(240*12, dtype='f').reshape(240,12),columns=Station_list)
eq_local_quantized8_values= pd.DataFrame(np.zeros(240*12, dtype='f').reshape(240,12),columns=Station_list)
km_local_quantized8_values= pd.DataFrame(np.zeros(240*12, dtype='f').reshape(240,12),columns=Station_list)
eq_local_edges8= pd.DataFrame(np.zeros(9*12, dtype='f').reshape(9,12),columns=Station_list)
km_local_edges8= pd.DataFrame(np.zeros(9*12, dtype='f').reshape(9,12),columns=Station_list)
for Station in Station_list:
    eq_local_quantized8[Station],eq_local_quantized8_values[Station],NA,eq_local_edges8[Station]=eqdistant_local_quantization(originaldata[Station],8)
    km_local_quantized8[Station],km_local_quantized8_values[Station],NA,km_local_edges8[Station]=km_local_quantization(originaldata[Station],8)
    

# Summary of Results 

# Figure4: Histograms of different quantization methods for Station12

In [ ]:
s=11 # station number >> 0 to 11
from sklearn.metrics import mean_squared_error
x = pd.concat([originaldata.iloc[:,s]],axis=1)
y = pd.concat([eq_global_quantized8_values.iloc[:,s]],axis=1)
z = pd.concat([eq_global_quantized14_values.iloc[:,s]],axis=1)
w = pd.concat([km_global_quantized14_values.iloc[:,s]],axis=1)
p = pd.concat([floor_global_quantized14_values.iloc[:,s]],axis=1)

biny=eq_global_edges8
binz=eq_global_edges14
binw=km_global_edges14
binp=floor_global_edges14

x=x.to_numpy()
y=y.to_numpy()
z=z.to_numpy()
w=w.to_numpy()
p=p.to_numpy()


left, width = 0.1, 0.5
bottom, height = 0.1, 0.5
spacing = 0.005

rect_scatter = [left, bottom, width, height]
rect_histy = [left + width + spacing, bottom, 0.15,height]
rect_histz = [left + width + spacing+spacing+ 0.15, bottom, 0.15, height]
rect_histw = [left + width + spacing+2*spacing+ 0.3, bottom, 0.15, height]
rect_histp = [left + width + spacing+3*spacing+ 0.45, bottom, 0.15, height]

# start with a rectangular Figure
fig=plt.figure(figsize=(14, 13))
ax_scatter = plt.axes(rect_scatter)
ax_scatter.tick_params(direction='in', bottom=True, left=True)
ax_histy = plt.axes(rect_histy)
ax_histy.tick_params(direction='in', labelleft=False)
ax_histz = plt.axes(rect_histz)
ax_histz.tick_params(direction='in', labelleft=False)
ax_histw = plt.axes(rect_histw)
ax_histw.tick_params(direction='in', labelleft=False)
ax_histp = plt.axes(rect_histp)
ax_histp.tick_params(direction='in', labelleft=False)

si=18
lim_min=np.min([y,z,w,p])
lim_max=np.max([y,z,w,p])
RMSE_eq_global8=math.sqrt(mean_squared_error(x, y))
RMSE_eq_global14=math.sqrt(mean_squared_error(x, z))
RMSE_eq_kmeans_global14=math.sqrt(mean_squared_error(x, w))
RMSE_eq_floorglobal14=math.sqrt(mean_squared_error(x, p))

# the scatter plot:
ax_scatter.scatter(x, y,s=100,c='g',linewidths=.01,marker="+",label = 'original VS eq global8;            RMSE={:.4}'.format(RMSE_eq_global8))
ax_scatter.scatter(x, z,s=100,c='r',linewidths=.01,marker="o",label = 'original VS eq global14;           RMSE={:.4}'.format(RMSE_eq_global14))
ax_scatter.scatter(x, w,s=100,c='darkorange',linewidths=.01,marker="*",label = 'original VS kmeans global14;   RMSE={:.4}'.format(RMSE_eq_kmeans_global14))
ax_scatter.scatter(x, p,s=100,c='c',linewidths=.01,marker="p",label = 'original VS floor global14;        RMSE={:.4}'.format(RMSE_eq_floorglobal14))
ax_scatter.set_ylabel('quantized center values',fontsize=si)
ax_scatter.set_xlabel('original data',fontsize=si)
ax_scatter.plot(ax_scatter.get_xlim(), ax_scatter.get_xlim(), 'k-', color = 'black')
ax_scatter.tick_params(direction='in', labelsize=si)


ax_histy.hist(y, bins=biny, orientation='horizontal',color='g',density=1)
ax_histz.hist(z, bins=binz, orientation='horizontal',color='r',density=1)
ax_histw.hist(w, bins=binw, orientation='horizontal',color='darkorange',density=1)
ax_histp.hist(p, bins=binp, orientation='horizontal',color='c',density=1)


off_set=75
ax_scatter.set_xlim(lim_min-off_set, lim_max+2*off_set)
ax_scatter.set_ylim(lim_min-off_set,lim_max+2*off_set)
ax_histy.set_ylim(ax_scatter.get_xlim())
ax_histz.set_ylim(ax_scatter.get_xlim())
ax_histw.set_ylim(ax_scatter.get_xlim())
ax_histp.set_ylim(ax_scatter.get_xlim())



ax_histy.xaxis.set_major_locator(plt.MaxNLocator(3))
f = mticker.ScalarFormatter(useOffset=True, useMathText=True)
g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.10e' % x))
ax_histy.xaxis.set_major_formatter(mticker.FuncFormatter(g))
ax_histy.tick_params(labelsize=si-4)
ax_histy.tick_params(bottom=True, top=False, left=False, right=False)

ax_histz.xaxis.set_major_locator(plt.MaxNLocator(3))
f = mticker.ScalarFormatter(useOffset=False, useMathText=False)
g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.10e' % x))
ax_histz.xaxis.set_major_formatter(mticker.FuncFormatter(g))
ax_histz.tick_params(labelsize=si-4)
ax_histz.tick_params(bottom=True, top=False, left=False, right=False)

ax_histw.xaxis.set_major_locator(plt.MaxNLocator(3))
f = mticker.ScalarFormatter(useOffset=False, useMathText=True)
g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.10e' % x))
ax_histw.xaxis.set_major_formatter(mticker.FuncFormatter(g))
ax_histw.tick_params(labelsize=si-4)
ax_histw.tick_params(bottom=True, top=False, left=False, right=False)

ax_histp.xaxis.set_major_locator(plt.MaxNLocator(3))
f = mticker.ScalarFormatter(useOffset=False, useMathText=True)
g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.10e' % x))
ax_histp.xaxis.set_major_formatter(mticker.FuncFormatter(g))
ax_histp.tick_params(labelsize=si-4)
ax_histp.tick_params(bottom=True, top=False, left=False, right=False)

ax_histy.set_title(label='a) eq_global8 \n H=1.53 bits',fontsize=si)
ax_histz.set_title(label='b) eq_global14\n H=2.25 bits',fontsize=si)
ax_histw.set_title(label='c) Kmeans_global14\n H=3.31 bits',fontsize=si)
ax_histp.set_title(label='d) floor_global14\n H=2.47 bits',fontsize=si)

ax_histy.set_xlabel('probability density',fontsize=si)
ax_histz.set_xlabel('probability density',fontsize=si)
ax_histw.set_xlabel('probability density',fontsize=si)
ax_histp.set_xlabel('probability density',fontsize=si)

ax_scatter.set_title('Station{}'.format(s+1),position=[left+2*height, bottom + 2.1*height ] ,fontsize=18,fontweight="bold")
ax_scatter.legend(loc='upper left',fontsize=si-4)
ax_scatter.grid()
ax_histy.grid()
ax_histz.grid()
ax_histw.grid()
ax_histp.grid()

plt.show()
fig.savefig("S12_scatter.png",dpi=500, bbox_inches="tight",transparent=10)


# Figure5: Histograms of different quantization methods for Station1

In [ ]:
s=0 # station number >> 0 to 11
from sklearn.metrics import mean_squared_error
x = pd.concat([originaldata.iloc[:,s]],axis=1)
y = pd.concat([eq_global_quantized8_values.iloc[:,s]],axis=1)
z = pd.concat([eq_global_quantized14_values.iloc[:,s]],axis=1)
w = pd.concat([km_global_quantized14_values.iloc[:,s]],axis=1)
p = pd.concat([floor_global_quantized14_values.iloc[:,s]],axis=1)

biny=eq_global_edges8
binz=eq_global_edges14
binw=km_global_edges14
binp=floor_global_edges14

x=x.to_numpy()
y=y.to_numpy()
z=z.to_numpy()
w=w.to_numpy()
p=p.to_numpy()


left, width = 0.1, 0.5
bottom, height = 0.1, 0.5
spacing = 0.005

rect_scatter = [left, bottom, width, height]
rect_histy = [left + width + spacing, bottom, 0.15,height]
rect_histz = [left + width + spacing+spacing+ 0.15, bottom, 0.15, height]
rect_histw = [left + width + spacing+2*spacing+ 0.3, bottom, 0.15, height]
rect_histp = [left + width + spacing+3*spacing+ 0.45, bottom, 0.15, height]

# start with a rectangular Figure
fig=plt.figure(figsize=(14, 13))
ax_scatter = plt.axes(rect_scatter)
ax_scatter.tick_params(direction='in', bottom=True, left=True)
ax_histy = plt.axes(rect_histy)
ax_histy.tick_params(direction='in', labelleft=False)
ax_histz = plt.axes(rect_histz)
ax_histz.tick_params(direction='in', labelleft=False)
ax_histw = plt.axes(rect_histw)
ax_histw.tick_params(direction='in', labelleft=False)
ax_histp = plt.axes(rect_histp)
ax_histp.tick_params(direction='in', labelleft=False)

si=18
lim_min=np.min([y,z,w,p])
lim_max=np.max([y,z,w,p])
RMSE_eq_global8=math.sqrt(mean_squared_error(x, y))
RMSE_eq_global14=math.sqrt(mean_squared_error(x, z))
RMSE_eq_kmeans_global14=math.sqrt(mean_squared_error(x, w))
RMSE_eq_floorglobal14=math.sqrt(mean_squared_error(x, p))

# the scatter plot:
ax_scatter.scatter(x, y,s=100,c='g',linewidths=.01,marker="+",label = 'original VS eq global8;            RMSE={:.5}'.format(RMSE_eq_global8))
ax_scatter.scatter(x, z,s=100,c='r',linewidths=.01,marker="o",label = 'original VS eq global14;           RMSE={:.4}'.format(RMSE_eq_global14))
ax_scatter.scatter(x, w,s=100,c='darkorange',linewidths=.01,marker="*",label = 'original VS kmeans global14;   RMSE={:.3}'.format(RMSE_eq_kmeans_global14))
ax_scatter.scatter(x, p,s=100,c='c',linewidths=.01,marker="p",label = 'original VS floor global14;        RMSE={:.4}'.format(RMSE_eq_floorglobal14))
ax_scatter.set_ylabel('quantized center values',fontsize=si)
ax_scatter.set_xlabel('original data',fontsize=si)
ax_scatter.plot(ax_scatter.get_xlim(), ax_scatter.get_xlim(), 'k-', color = 'black')
ax_scatter.tick_params(direction='in', labelsize=si)


ax_histy.hist(y, bins=biny, orientation='horizontal',color='g',density=1)
ax_histz.hist(z, bins=binz, orientation='horizontal',color='r',density=1)
ax_histw.hist(w, bins=binw, orientation='horizontal',color='darkorange',density=1)
ax_histp.hist(p, bins=binp, orientation='horizontal',color='c',density=1)


off_set=10
ax_scatter.set_xlim(lim_min-off_set, lim_max+100+off_set)
ax_scatter.set_ylim(lim_min-off_set,lim_max+100+off_set)
ax_histy.set_ylim(ax_scatter.get_xlim())
ax_histz.set_ylim(ax_scatter.get_xlim())
ax_histw.set_ylim(ax_scatter.get_xlim())
ax_histp.set_ylim(ax_scatter.get_xlim())
ax_scatter.plot(ax_scatter.get_xlim(), ax_scatter.get_xlim(), 'k-', color = 'black')



ax_histy.xaxis.set_major_locator(plt.MaxNLocator(3))
f = mticker.ScalarFormatter(useOffset=True, useMathText=True)
g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.10e' % x))
ax_histy.xaxis.set_major_formatter(mticker.FuncFormatter(g))
ax_histy.tick_params(labelsize=si-6)
ax_histy.tick_params(bottom=True, top=False, left=False, right=False)

ax_histz.xaxis.set_major_locator(plt.MaxNLocator(3))
f = mticker.ScalarFormatter(useOffset=False, useMathText=False)
g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.10e' % x))
ax_histz.xaxis.set_major_formatter(mticker.FuncFormatter(g))
ax_histz.tick_params(labelsize=si-6)
ax_histz.tick_params(bottom=True, top=False, left=False, right=False)

ax_histw.xaxis.set_major_locator(plt.MaxNLocator(3))
f = mticker.ScalarFormatter(useOffset=False, useMathText=True)
g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.10e' % x))
ax_histw.xaxis.set_major_formatter(mticker.FuncFormatter(g))
ax_histw.tick_params(labelsize=si-6)
ax_histw.tick_params(bottom=True, top=False, left=False, right=False)

ax_histp.xaxis.set_major_locator(plt.MaxNLocator(3))
f = mticker.ScalarFormatter(useOffset=False, useMathText=True)
g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.10e' % x))
ax_histp.xaxis.set_major_formatter(mticker.FuncFormatter(g))
ax_histp.tick_params(labelsize=si-6)
ax_histp.tick_params(bottom=True, top=False, left=False, right=False)


ax_histy.set_title(label='a) eq_global8 \n H=0 bits',fontsize=si)
ax_histz.set_title(label='b) eq_global14\n H=0 bits',fontsize=si)
ax_histw.set_title(label='c) Kmeans_global14\n H=0.57 bits',fontsize=si)
ax_histp.set_title(label='d) floor_global14\n H=0.1 bits',fontsize=si)

ax_histy.set_xlabel('probability density',fontsize=si)
ax_histz.set_xlabel('probability density',fontsize=si)
ax_histw.set_xlabel('probability density',fontsize=si)
ax_histp.set_xlabel('probability density',fontsize=si)

ax_scatter.set_title('Station{}'.format(s+1),position=[left+2*height, bottom + 2.1*height ] ,fontsize=18,fontweight="bold")
ax_scatter.legend(loc='upper left',fontsize=si-4)
ax_scatter.grid()
ax_histy.grid()
ax_histz.grid()
ax_histw.grid()
ax_histp.grid()

plt.show()
fig.savefig("S1_scatter.png",dpi=500, bbox_inches="tight",transparent=10)


# Figure6: Number and center of bins in different quantization methods for Station1

In [ ]:
s=0 # station number >> 0 to 11
df1=pd.concat([originaldata.iloc[:,s],eq_local_quantized8_values.iloc[:,s],km_local_quantized8_values.iloc[:,s],
              eq_global_quantized8_values.iloc[:,s],km_global_quantized8_values.iloc[:,s],
              eq_global_quantized14_values.iloc[:,s],km_global_quantized14_values.iloc[:,s],floor_global_quantized14_values.iloc[:,s]],axis=1)
df1.columns = ['originaldata', 'eq_local8','Kmeans_local8','eq_global8','Kmeans_global8','eq_global14','Kmeans_global14','floor14']
df1.head()

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(30,10))
sns.set(style="whitegrid",font_scale=1)
sns.set_context("notebook", font_scale=2.1)

yposlist = df1.max()
yposlist1=yposlist['eq_global8']
yposlist2=yposlist['eq_global14']
circle_rad = 15
ax.plot(3,yposlist1,'o',
        ms=circle_rad * 2, mec='r', mfc='none', mew=4)

ax.plot(5, yposlist2, 'o',
        ms=circle_rad * 2, mec='r', mfc='none', mew=4)
sns.despine(left=True, bottom=True)
sns.violinplot(data=df1 ,cut=0,scale= 'count',inner='point')
ax.set_title(r'Station{}'.format(s+1),fontweight='bold',fontsize=30)

stringlist1 = 'Only one bin'
xposlist1 = 3
ax.text(xposlist1, yposlist1+15, stringlist1,
horizontalalignment='center',
verticalalignment='top',fontsize=25,fontweight='bold')
ax.text(xposlist1+2, yposlist2+15, stringlist1,
horizontalalignment='center',
verticalalignment='top',fontsize=25,fontweight='bold')
yposlist2 = df1.max()+10
yposlist2[3]=yposlist2[3]-20
yposlist2[5]=yposlist2[5]-20
xposlist2 = range(len(yposlist))
stringlist2 = ['','H(S1)=0.91', 'H(S1)=2.12','H(S1)=0','H(S1)=0.15','H(S1)=0',
              'H(S1)=0.57','H(S1)=0.1']

for i in range(len(stringlist2)):
    ax.text(xposlist2[i], yposlist2[i], stringlist2[i],
        horizontalalignment='center',
        verticalalignment='top',fontsize=25,fontweight='bold')

ax.set_ylabel('quantized streamflow',fontweight='bold',fontsize=30)   
fig.savefig("quantz_S1_entropy",dpi=500, bbox_inches="tight", pad_inches=.5,transparent=True)

# Figure7: Number and center of bins in different quantization methods for Station12

In [ ]:
s=11 # station number >> 0 to 11
df1=pd.concat([originaldata.iloc[:,s],eq_local_quantized8_values.iloc[:,s],km_local_quantized8_values.iloc[:,s],
              eq_global_quantized8_values.iloc[:,s],km_global_quantized8_values.iloc[:,s],
              eq_global_quantized14_values.iloc[:,s],km_global_quantized14_values.iloc[:,s],floor_global_quantized14_values.iloc[:,s]],axis=1)
df1.columns = ['originaldata', 'eq_local8','Kmeans_local8','eq_global8','Kmeans_global8','eq_global14','Kmeans_global14','floor14']
df1.head()

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(30,10))
sns.set(style="whitegrid",font_scale=1)
sns.set_context("notebook", font_scale=2.1)


sns.despine(left=True, bottom=True)
sns.violinplot(data=df1 ,cut=0,scale= 'count',inner='point')
ax.set_title(r'Station{}'.format(s+1),fontweight='bold',fontsize=30)

yposlist = df1.max()+130
xposlist = range(len(yposlist))
stringlist = ['','H=1.53', 'H=2.28','H=1.53','H=2.58','H=2.25',
              'H=3.31','H=2.47']

for i in range(len(stringlist)):
    ax.text(xposlist[i], yposlist[i], stringlist[i],
        horizontalalignment='center',
        verticalalignment='top',fontsize=25,fontweight='bold')

ax.set_ylabel('quantized streamflow',fontweight='bold',fontsize=30)   

fig.savefig("quantz_S12_entropy",dpi=700, bbox_inches="tight", pad_inches=.5,transparent=True)

# Table1: Comparison of data representativeness with Surprise score

In [ ]:
from Info_theory_function_lists import likelihood_calculator_quantized_data
Table1= pd.DataFrame(np.zeros(7*12, dtype='f').reshape(7,12),index=['floor14','km_global14','eq_global14','km_global8','eq_global8','km_local8','eq_local8',]
                  ,columns=['Station1','Station2','Station3','Station4','Station5','Station6','Station7','Station8','Station9','Station10','Station11','Station12',])
for i in np.arange(12):
    Table1.iloc[0,i]=likelihood_calculator_quantized_data(floor_global_quantized14_values.iloc[:,i],floor_global_edges14)
    Table1.iloc[1,i]=likelihood_calculator_quantized_data(km_global_quantized14_values.iloc[:,i],km_global_edges14)
    Table1.iloc[2,i]=likelihood_calculator_quantized_data(eq_global_quantized14_values.iloc[:,i],eq_global_edges14)
    Table1.iloc[3,i]=likelihood_calculator_quantized_data(km_global_quantized8_values.iloc[:,i],km_global_edges8)
    Table1.iloc[4,i]=likelihood_calculator_quantized_data(eq_global_quantized8_values.iloc[:,i],eq_global_edges8)
    Table1.iloc[5,i]=likelihood_calculator_quantized_data(km_local_quantized8_values.iloc[:,i],km_local_edges8.iloc[:,i])
    Table1.iloc[6,i]=likelihood_calculator_quantized_data(eq_local_quantized8_values.iloc[:,i],eq_local_edges8.iloc[:,i])

display('======================================Table1: Comparison of data representativeness with Surprise score ==================================')
display(Table1)

# Table2: Comparison of data representativeness with divergence score

In [ ]:
from Info_theory_function_lists import projection_to_fine_prob_grid
from Info_theory_function_lists import Kullback_Leibler_divergence_score


Table2= pd.DataFrame(np.zeros(5*12, dtype='f').reshape(5,12),index=['floor14','km_global14','eq_global14','km_global8','eq_global8']
                  ,columns=['Station1','Station2','Station3','Station4','Station5','Station6','Station7','Station8','Station9','Station10','Station11','Station12',])

bin_num=300 # number of bins in fine grid ==>> to project probabilities into common grid for two quantization methods
max_value=originaldata.max().max()
min_value=originaldata.min().min()
Station_list=originaldata.columns


for i in np.arange(12):
    P=projection_to_fine_prob_grid(eq_local_quantized8_values.iloc[:,i],eq_local_edges8.iloc[:,i].values,max_value,min_value,bin_num)[0]
    Q=projection_to_fine_prob_grid(floor_global_quantized14_values.iloc[:,i],floor_global_edges14,max_value,min_value,bin_num)[0]
    Table2.iloc[0,i]=Kullback_Leibler_divergence_score(P,Q)
    
    P=projection_to_fine_prob_grid(eq_local_quantized8_values.iloc[:,i],eq_local_edges8.iloc[:,i].values,max_value,min_value,bin_num)[0]
    Q=projection_to_fine_prob_grid(km_global_quantized14_values.iloc[:,i],km_global_edges14,max_value,min_value,bin_num)[0]
    Table2.iloc[1,i]=Kullback_Leibler_divergence_score(P,Q)
                                   
    P=projection_to_fine_prob_grid(eq_local_quantized8_values.iloc[:,i],eq_local_edges8.iloc[:,i].values,max_value,min_value,bin_num)[0]
    Q=projection_to_fine_prob_grid(eq_global_quantized14_values.iloc[:,i],eq_global_edges14,max_value,min_value,bin_num)[0]
    Table2.iloc[2,i]=Kullback_Leibler_divergence_score(P,Q)
    
    P=projection_to_fine_prob_grid(eq_local_quantized8_values.iloc[:,i],eq_local_edges8.iloc[:,i].values,max_value,min_value,bin_num)[0]
    Q=projection_to_fine_prob_grid(km_global_quantized8_values.iloc[:,i],km_global_edges8,max_value,min_value,bin_num)[0]
    Table2.iloc[3,i]=Kullback_Leibler_divergence_score(P,Q)
                                   
    P=projection_to_fine_prob_grid(eq_local_quantized8_values.iloc[:,i],eq_local_edges8.iloc[:,i].values,max_value,min_value,bin_num)[0]
    Q=projection_to_fine_prob_grid(eq_global_quantized8_values.iloc[:,i],eq_global_edges8,max_value,min_value,bin_num)[0]
    Table2.iloc[4,i]=Kullback_Leibler_divergence_score(P,Q)    
    
#take average   
Table2.insert(0, "Average Score", Table2.mean(axis=1), True) 

display('==========================Table2: Comparison of data representativeness with divergence score from eq_local8 =====================')
display(Table2)

# Table3: Optimal gauges order with different quantization methods and objective functions 

In [ ]:
from Info_theory_function_lists import greedy_optimizer_maxJE
from Info_theory_function_lists import greedy_optimizer_minT
Table3_part1_maxJE= pd.DataFrame(np.zeros(7*12, dtype='f').reshape(7,12).astype('int64'),index=['floor14','km_global14','eq_global14','km_global8','eq_global8','km_local8','eq_local8',]
                  ,columns=['Ranked1','Ranked2','Ranked3','Ranked4','Ranked5','Ranked6','Ranked7','Ranked8','Ranked9','Ranked10','Ranked11','Ranked12',])

Table3_part2_minT= pd.DataFrame(np.zeros(7*12, dtype='f').reshape(7,12).astype('int64'),index=['floor14','km_global14','eq_global14','km_global8','eq_global8','km_local8','eq_local8',]
                  ,columns=['Ranked1','Ranked2','Ranked3','Ranked4','Ranked5','Ranked6','Ranked7','Ranked8','Ranked9','Ranked10','Ranked11','Ranked12',])


Table3_part1_maxJE.iloc[0,:]=greedy_optimizer_maxJE(floor_global_quantized14)
Table3_part1_maxJE.iloc[1,:]=greedy_optimizer_maxJE(km_global_quantized14)
Table3_part1_maxJE.iloc[2,:]=greedy_optimizer_maxJE(eq_global_quantized14)
Table3_part1_maxJE.iloc[3,:]=greedy_optimizer_maxJE(km_global_quantized8)
Table3_part1_maxJE.iloc[4,:]=greedy_optimizer_maxJE(eq_global_quantized8)
Table3_part1_maxJE.iloc[5,:]=greedy_optimizer_maxJE(km_local_quantized8)
Table3_part1_maxJE.iloc[6,:]=greedy_optimizer_maxJE(eq_local_quantized8)

Table3_part2_minT.iloc[0,:]=greedy_optimizer_minT(floor_global_quantized14)
Table3_part2_minT.iloc[1,:]=greedy_optimizer_minT(km_global_quantized14)
Table3_part2_minT.iloc[2,:]=greedy_optimizer_minT(eq_global_quantized14)
Table3_part2_minT.iloc[3,:]=greedy_optimizer_minT(km_global_quantized8)
Table3_part2_minT.iloc[4,:]=greedy_optimizer_minT(eq_global_quantized8)
Table3_part2_minT.iloc[5,:]=greedy_optimizer_minT(km_local_quantized8)
Table3_part2_minT.iloc[6,:]=greedy_optimizer_minT(eq_local_quantized8)


display('=================================================Table3 part1: maxJE method =================================================')
display(Table3_part1_maxJE)
display('================================================== Table3 part2: minT method =================================================')
display(Table3_part2_minT)

# Table4: Evolution of multivariate joint entropy based on maxJE method 

In [ ]:
from Info_theory_function_lists import info_stats_merging_operator_multiple_varibles

Table4= pd.DataFrame(np.zeros(7*12, dtype='f').reshape(7,12),index=['floor14','km_global14','eq_global14','km_global8','eq_global8','km_local8','eq_local8',]
                  ,columns=['multi_dim_1','multi_dim_2','multi_dim_3','multi_dim_4','multi_dim_5','multi_dim_6','multi_dim_7','multi_dim_8','multi_dim_9','multi_dim_10','multi_dim_11','multi_dim_12',])

for i in np.arange(12):
    Table4.iloc[0,i]=info_stats_merging_operator_multiple_varibles(floor_global_quantized14,Table3_part1_maxJE.iloc[0,0:i+1])[0]
    Table4.iloc[1,i]=info_stats_merging_operator_multiple_varibles(km_global_quantized14,Table3_part1_maxJE.iloc[1,0:i+1])[0]
    Table4.iloc[2,i]=info_stats_merging_operator_multiple_varibles(eq_global_quantized14,Table3_part1_maxJE.iloc[2,0:i+1])[0]
    Table4.iloc[3,i]=info_stats_merging_operator_multiple_varibles(km_global_quantized8,Table3_part1_maxJE.iloc[3,0:i+1])[0]
    Table4.iloc[4,i]=info_stats_merging_operator_multiple_varibles(eq_global_quantized8,Table3_part1_maxJE.iloc[4,0:i+1])[0]
    Table4.iloc[5,i]=info_stats_merging_operator_multiple_varibles(km_local_quantized8,Table3_part1_maxJE.iloc[5,0:i+1])[0]
    Table4.iloc[6,i]=info_stats_merging_operator_multiple_varibles(eq_local_quantized8,Table3_part1_maxJE.iloc[6,0:i+1])[0]

display('================================Evolution of multivariate joint entropy based on ranking obtained by maxJE objective function ===============================')
display(Table4)